# 3-channel image compiler for EPMA elemental maps of petrographic thin sections

This notebook contains a script with step-by-step explanations to create a widget that stacks three 8-bit grayscale `.tiff` files into a single 3-channel RGB image and saves it your local drive as a new `.tiff` file. 

The first three code blocks were provided by Akshay Mehra.

## Overview
The purpose of this notebook is to composite EPMA elemental map data into 3-channel RGB images. 

training and testing datasets for a machine learning model that classifies mineral grains based on elemental compositions at each microprobe measurement point, or pixel
mineral assemblages based on the primary 
produce 3-channel RGB composite images for 
a machine learning 

#### Steps
1. make widget
2. create RGB `.tiffs`
3. ...
4. ..

### Data
Each thin section has 10 elemental maps `.tiff` files, each for a different element. 
Thin section 1 images are 703 x 1100 pixels
Thin section 2 images are 575 x 1026 pixels
Thin section 3 images are 695 x 1152 pixels
Thin section 3 images are 695 x 1152 pixels


### This is going to need to be written after I have worked on this some more. 

## Creating 

In [13]:

import numpy as np                      
import os                               # to work with filepaths and local directories
import matplotlib.pyplot as plt         # to display the output images in the notebook

# Python image library (PIL) lets you access .tiff files
from PIL import Image

# to create and add functionality to a widget
import ipywidgets as widgets

# to display the widget
from IPython.display import display

# to save arrays to .tiff files
import imageio

In [20]:
# LOADING THE TIFFS
# Load in a single image
singleImage = Image.open('/Users/jonathanlindenmann/jonspace/000_aut_23/ESS469/Git/ML-Geo-Pixel-Poppers/Aikin_Data/78.7-10-1_Hot/RAW/UGG-W3-87.7-10.1-Full_Al Ka_Al Ka EDS.Tiff')
# Turn this image into an array
singleImageArray = np.asarray(singleImage)
print(singleImageArray)


[[ 81.  90.  72. ... 382. 372. 366.]
 [ 94. 101.  88. ... 660. 591. 382.]
 [ 84.  79.  89. ... 587. 628. 741.]
 ...
 [413. 427. 400. ...  77. 106. 113.]
 [452. 434. 470. ...  95.  82.  81.]
 [425. 460. 390. ... 102.  88.  92.]]


## 

1. Load the .tiff files and turn them into arrays.

## Break down of the code provided by Akshay:

We have four thin sections, and each one has ten .tiffs in a 'RAW' folder. We want to create a stack of all ten .tiffs for each thin section in an array, then make a list object that contains all four image stacks, together with a name for each. 

1. Create the list `thinSections`. This is a `dict` object that contains the thin section `name` and `stack` (the root folder )
2. Create a loop that iterates over every directory in the parent folder with the name 'RAW' and extracts the .tiffs within into a 3D array, then adds the image stack to the `thinSections` list object:
    2.1 Navigates to a directory called 'RAW'
    
    2.1.1 In the 'RAW' directory, use the name of the folder the 'RAW' folder is in as the `thinSectionName` and adds it to the thinSections object as a key:`name` `dict` object.
        
    2.1.2 Find the number of files in the 'RAW' folder. 
        
    2.1.3 For all 'RAW' folders that have files in them, 
            
    2.1.3.1 Create boolean that logs whether or not the image stack is complete. 
    
    2.1.3.2 Create an empty array for the image stack. 
    
    2.1.3.3 Count the number of image stacks/RAW folders being iterated over.
    
    2.1.4 For each .tiff, open the file, turn it into an array, and add it to the `tsArray`, which is the loop's working image stack, at the layer specified by section count. The section count indexes each 2D layer by counting the layers.
            
    Refer to the code to understand the rest. 
            

        

In [40]:
# We know that we can turn one image into an array...can we stack all the images for a thin section into one, 3D array?

# Create an array that will contain each thin section

thinSections = [] #empty LIST for the all stacked thin section arrays

iteration = 0

# Okay, for every folder in our parent folder

for root, dirs, files in os.walk('.//'):
    # './/' means the CURRENT working directory. In this case, it is the one that the jupyter notebook is stored in (perfect!)
       
        # Does the directory contain the word RAW?
        if "RAW" in root:
            # If so, what is the name of the thin section?
            thinSectionName = os.path.basename(os.path.dirname(root))
            thinSections.append(dict({'name': thinSectionName}))
            numberOfFiles = len(files)
           
            if numberOfFiles > 0:
                arraySet = False #
                tsArray = [] #the array of stacked thin sections
                sectionCount = 0
                
                for file in files:
                    tsImage = Image.open(os.path.join(root, file))
                    singleImageArray = np.asarray(tsImage)
                    print(singleImageArray.shape)
                    tsImageName = file
                    print(tsImageName)
                    if not arraySet:
                        tsShape = singleImageArray.shape
                        tsArray = np.empty([tsShape[0], tsShape[1], numberOfFiles])
                        arraySet = True
                    tsArray[:, :, sectionCount] = singleImageArray
                    sectionCount = sectionCount + 1
                thinSections[iteration]['stack'] = tsArray # add the array as the 'stack' key's value to the thinSections list
                                                           # to the list item corresponding with the iteration (thin section 1, 2, 3, or ,4.)
            iteration = iteration + 1
            #for file in files:
              #print(os.path.join(root, file)) #prints the filepath (each iteration, so each all .tiffs from all thin sections)

#these print commands are for troubleshooting.
#print(thinSectionName)
#print("tsArray.shape:", tsArray.shape)
#print ("iteration", iteration)
#print("sectionCount:", sectionCount)
print("thinSections:", thinSections)
#print("length of thinSections:", len(thinSections))



(703, 1100)
CC-84.7-R21-NA2-1_full_Fe Ka_Fe Ka EDS.Tiff
(703, 1100)
CC-84.7-R21-NA2-1_full_Y  La_Y  La (Sp 4).Tiff
(703, 1100)
CC-84.7-R21-NA2-1_full_Ca Ka_Ca Ka EDS.Tiff
(703, 1100)
CC-84.7-R21-NA2-1_full_Zr La_Zr La (Sp 3).Tiff
(703, 1100)
CC-84.7-R21-NA2-1_full_Ti Ka_Ti Ka (Sp 5).Tiff
(703, 1100)
CC-84.7-R21-NA2-1_full_Al Ka_Al Ka EDS.Tiff
(703, 1100)
CC-84.7-R21-NA2-1_full_Ce La_Ce La (Sp 2).Tiff
(703, 1100)
CC-84.7-R21-NA2-1_full_Mg Ka_Mg Ka (Sp 1).Tiff
(703, 1100)
CC-84.7-R21-NA2-1_full_K  Ka_K  Ka EDS.Tiff
(575, 1026)
CC-84.7-NA2.2_Ti Ka_Ti Ka (Sp 5).Tiff
(575, 1026)
CC-84.7-NA2.2_Zr La_Zr La (Sp 3).Tiff
(575, 1026)
CC-84.7-NA2.2_K  Ka_K  Ka EDS.Tiff
(575, 1026)
CC-84.7-NA2.2_Y  La_Y  La (Sp 4).Tiff
(575, 1026)
CC-84.7-NA2.2_Fe Ka_Fe Ka EDS.Tiff
(575, 1026)
CC-84.7-NA2.2_Ca Ka_Ca Ka EDS.Tiff
(575, 1026)
CC-84.7-NA2.2_Mg Ka_Mg Ka (Sp 1).Tiff
(575, 1026)
CC-84.7-NA2.2_Ce La_Ce La (Sp 2).Tiff
(575, 1026)
CC-84.7-NA2.2_Al Ka_Al Ka EDS.Tiff
(695, 1152)
UGG-W3-87.7-10.1-Full_Ce La_Ce 

We now have `thinSections`, which is a list that contains 4 elements. Each element is a thin section, and contains `name: (name of thin section from root dir)` and `stack: (image stack of all the .tiffs of that thin section)`, in other words, it contains a string, and a 3D array: ten layers of 2D arrays.

Next we want to create a widget to work with these image stacks. 

In [41]:
thinsectionindex = 2
# Get the stack and its shape for the initially selected thin section
thisThinSection = thinSections[thinsectionindex]
stack = thisThinSection['stack']
stackShape = stack.shape


# Create a dropdown widget for thin section selection
thinSectionDropdown = widgets.Dropdown(
    options=range(len(thinSections)),
    description='Thin Section:'
)

# Create dropdown widgets for the three dimensions
channel1Dropdown = widgets.Dropdown(
    options=range(stackShape[2]),
    description='Channel 1:'
)

channel2Dropdown = widgets.Dropdown(
    options=range(stackShape[2]),
    description='Channel 2:'
)

channel3Dropdown = widgets.Dropdown(
    options=range(stackShape[2]),
    description='Channel 3:'
)

# Create a button widget to trigger visualization
visualizeBtn = widgets.Button(
    description='Visualize',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to visualize',
    icon='check'
)

# Create a button widget to save files
savefileBtn = widgets.Button(
    description='Save as .tiff',
    disabled=False,
    button_style='success',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to save selection to .tiff',
    icon=''
)

# Create an output widget to display the image
output = widgets.Output()

def visualize(b):

    # Get the selected thin section
    selectedThinSectionIndex = thinSectionDropdown.value
    
    thisThinSection = thinSections[selectedThinSectionIndex]

    thisStack = thisThinSection['stack']
    thisStackShape = thisStack.shape

    recastThinSection = np.empty([thisStackShape[0], thisStackShape[1], 3], dtype='int')
    

    recastThinSection[:,:,0] = thisStack[:,:,channel1Dropdown.value]
    recastThinSection[:,:,1] = thisStack[:,:,channel2Dropdown.value]
    recastThinSection[:,:,2] = thisStack[:,:,channel3Dropdown.value]

    # Display the image using Matplotlib
    with output:
        output.clear_output()
        plt.figure(figsize=(8, 6))
        plt.imshow(recastThinSection)  # You can change the colormap as needed
        plt.show()

def savefile(b):
    # Get the selected thin section
    selectedThinSectionIndex = thinSectionDropdown.value

    thisThinSection = thinSections[selectedThinSectionIndex]

    thisStack = thisThinSection['stack']
    thisStackShape = thisStack.shape

    recastThinSection = np.empty([thisStackShape[0], thisStackShape[1], 3], dtype='int')
    

    recastThinSection[:,:,0] = thisStack[:,:,channel1Dropdown.value]
    recastThinSection[:,:,1] = thisStack[:,:,channel2Dropdown.value]
    recastThinSection[:,:,2] = thisStack[:,:,channel3Dropdown.value]
    #turn the output array back into a .tiff (?)
    rgb_array_uint8 = np.clip(recastThinSection, 0, 255).astype(np.uint8)
    
    #save the file to the working directory
    imageio.imwrite('output_image.tiff', rgb_array_uint8)  


# Connect the button's click event to the visualization function
visualizeBtn.on_click(visualize) #when clicking the visualize button, run visualize(b)
savefileBtn.on_click(savefile)

# Display the widgets
display(thinSectionDropdown, channel1Dropdown, channel2Dropdown, channel3Dropdown, visualizeBtn, savefileBtn)
display(output)




    #for saving the image...
    #rgb_array_uint8 = np.clip(recastThinSection, 0, 255).astype(np.uint8)
    #imageio.imwrite('output_image.tiff', rgb_array_uint8)


Dropdown(description='Thin Section:', options=(0, 1, 2, 3), value=0)

Dropdown(description='Channel 1:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0)

Dropdown(description='Channel 2:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0)

Dropdown(description='Channel 3:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9), value=0)

Button(button_style='success', description='Visualize', icon='check', style=ButtonStyle(), tooltip='Click to v…

Button(button_style='success', description='Save as .tiff', style=ButtonStyle(), tooltip='Click to save select…

Output()

In [ ]:
1. Add the names of each layer to the dropdown lists for each channel. (customized dropdown fields) Add a 'display vector'.
2. 

3. training data?
